In [13]:
%pip install git+https://github.com/MarcusLoppe/meshgpt-pytorch.git
%pip install matplotlib

  Cloning https://github.com/MarcusLoppe/meshgpt-pytorch.git to /tmp/pip-req-build-5ilwomrd
  Running command git clone --filter=blob:none --quiet https://github.com/MarcusLoppe/meshgpt-pytorch.git /tmp/pip-req-build-5ilwomrd
  Resolved https://github.com/MarcusLoppe/meshgpt-pytorch.git to commit b5e74674973de4a97431561d7723d9b29629a695
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
from pathlib import Path 
import gc    
import torch
import os
import torch  
from meshgpt_pytorch import (
    MeshTransformerTrainer,
    MeshAutoencoderTrainer,
    MeshAutoencoder,
    MeshTransformer,MeshDataset
)
from meshgpt_pytorch.data import ( 
    derive_face_edges_from_faces
)   

In [15]:
%cd /root/text-to-mesh

/root/text-to-mesh


In [16]:
autoencoder = MeshAutoencoder( 
        decoder_dims_through_depth =  (128,) * 6 + (192,) * 12 + (256,) * 24 + (384,) * 6,    
        dim_codebook = 192,  
        dim_area_embed = 16,
        dim_coor_embed = 16, 
        dim_normal_embed = 16,
        dim_angle_embed = 8,
    
    attn_decoder_depth  = 4,
    attn_encoder_depth = 2
    ).to("cuda")     

In [17]:
dataset = MeshDataset.load("./objverse_250f_45.9M_3086_labels_53730_10_min_x1_aug.npz")  
dataset2 = MeshDataset.load("./objverse_250f_229.7M_3086_labels_268650_10_min_x5_aug.npz")
dataset.data.extend(dataset2.data)  
dataset2 = MeshDataset.load("./shapenet_250f_2.2M_84_labels_2156_10_min_x1_aug.npz")  
dataset.data.extend(dataset2.data)  
dataset2 = MeshDataset.load("./shapenet_250f_21.9M_84_labels_21560_10_min_x10_aug.npz")  
dataset.data.extend(dataset2.data) 
dataset.sort_dataset_keys()
print("length", len(dataset.data))

def format_value(value):
    if value >= 1_000_000_000:
        return f"{value / 1_000_000_000:.1f}B"
    elif value >= 1_000_000:
        return f"{value / 1_000_000:.1f}M"
    else:
        return f"{value}"

tokens = 0
for item in dataset.data:
    tokens += len(item['faces']) * 6 
total_tokens = format_value(tokens)
print("Tokens:", total_tokens)

[MeshDataset] Loaded 53730 entrys
[MeshDataset] Created from 53730 entrys
[MeshDataset] Loaded 268650 entrys
[MeshDataset] Created from 268650 entrys
[MeshDataset] Loaded 2156 entrys
[MeshDataset] Created from 2156 entrys
[MeshDataset] Loaded 21560 entrys
[MeshDataset] Created from 21560 entrys
length 346096
Tokens: 299.7M


In [18]:
pkg = torch.load("./checkpoints/mesh-autoencoder.ckpt.epoch_2_avg_loss_0.26642_recon_0.3762_commit_-0.5491.pt") 
autoencoder.load_state_dict(pkg['model'])
for param in autoencoder.parameters():
    param.requires_grad = True

In [20]:
batch_size=16 # The batch size should be max 64.
grad_accum_every =4
# So set the maximal batch size (max 64) that your VRAM can handle and then use grad_accum_every to create a effective batch size of 64, e.g  16 * 4 = 64
learning_rate = 1e-3 # Start with 1e-3 then at stagnation around 0.35, you can lower it to 1e-4.

autoencoder.commit_loss_weight = 0.2 # Set dependant on the dataset size, on smaller datasets, 0.1 is fine, otherwise try from 0.25 to 0.4.a
autoencoder_trainer = MeshAutoencoderTrainer(model =autoencoder ,warmup_steps = 10, dataset = dataset, num_train_steps=100,
                                             batch_size=batch_size,
                                             grad_accum_every = grad_accum_every,
                                             learning_rate = learning_rate,
                                             checkpoint_every_epoch=1)
 
loss = autoencoder_trainer.train(480, diplay_graph= True)   

Epoch 1/480:   0%|          | 2/21631 [00:00<43:00,  8.38it/s, commit_loss=-0.81, loss=0.187, recon_loss=0.349] 

Epoch 1/480: 100%|██████████| 21631/21631 [40:34<00:00,  8.88it/s, commit_loss=-0.656, loss=0.222, recon_loss=0.354]  


Epoch 1 average loss: 0.2550648029504936 recon loss: 0.3727: commit_loss -0.5880


Epoch 2/480: 100%|██████████| 21631/21631 [39:37<00:00,  9.10it/s, commit_loss=0.118, loss=0.471, recon_loss=0.447]   


Epoch 2 average loss: 0.23449984491546533 recon loss: 0.3665: commit_loss -0.6601


...

Epoch 9/480: 100%|██████████| 21631/21631 [39:53<00:00,  9.04it/s, commit_loss=-0.812, loss=0.183, recon_loss=0.345]  


Epoch 9 average loss: 0.19742232761713277 recon loss: 0.3536: commit_loss -0.7809          avg loss speed: 0.01477134139346431 epochs left: 6.94


Epoch 10/480: 100%|██████████| 21631/21631 [39:55<00:00,  9.03it/s, commit_loss=-0.643, loss=0.254, recon_loss=0.382]  


Epoch 10 average loss: 0.19880290955510385 recon loss: 0.3549: commit_loss -0.7807          avg loss speed: 0.007533533871931619 epochs left: 13.43


Epoch 11/480:   1%|          | 148/21631 [00:17<41:11,  8.69it/s, commit_loss=-0.401, loss=0.278, recon_loss=0.358]

In [ ]:
import datetime
import torch
now_utc = datetime.datetime.now(datetime.timezone.utc).isoformat()
now_safe = now_utc.replace(":", "_")
pkg = dict( model = autoencoder.state_dict(), )
filename = f"./MeshGPT-autoencoder_{now_safe}.pt"
torch.save(pkg, filename)

 
def combind_mesh_with_rows(path, meshes):
    all_vertices = []
    all_faces = []
    vertex_offset = 0
    translation_distance = 0.5  
    obj_file_content = ""
    
    for row, mesh in enumerate(meshes): 
        for r, faces_coordinates in enumerate(mesh): 
            numpy_data = faces_coordinates[0].cpu().numpy().reshape(-1, 3)  
            numpy_data[:, 0] += translation_distance * (r / 0.2 - 1)  
            numpy_data[:, 2] += translation_distance * (row / 0.2 - 1)  
        
            for vertex in numpy_data:
                all_vertices.append(f"v {vertex[0]} {vertex[1]} {vertex[2]}\n")
        
            for i in range(1, len(numpy_data), 3):
                all_faces.append(f"f {i + vertex_offset} {i + 1 + vertex_offset} {i + 2 + vertex_offset}\n")
        
            vertex_offset += len(numpy_data)
        
        obj_file_content = "".join(all_vertices) + "".join(all_faces)
     
    with open(path , "w") as file:
        file.write(obj_file_content)   
        
from meshgpt_pytorch import MeshAutoencoderTrainer, MeshAutoencoder, MeshDataset, mesh_render
import tqdm
min_mse, max_mse = float('inf'), float('-inf')
min_coords, min_orgs, max_coords, max_orgs = None, None, None, None
random_samples, random_samples_pred, all_random_samples = [], [], []
total_mse, sample_size = 0.0, 200

autoencoder = autoencoder.to("cuda")

for item in tqdm.tqdm(dataset.data[:sample_size]):  # Use tqdm.tqdm here
    item['faces'] = item['faces'].to("cuda")
    item['vertices'] = item['vertices'].to("cuda")
    item['face_edges'] = item['face_edges'].to("cuda")
    codes = autoencoder.tokenize(vertices=item['vertices'], faces=item['faces'], face_edges=item['face_edges']) 

    codes = codes.flatten().unsqueeze(0)
    codes = codes[:, :codes.shape[-1] // autoencoder.num_quantizers * autoencoder.num_quantizers] 

    coords, mask = autoencoder.decode_from_codes_to_faces(codes)
    orgs = item['vertices'][item['faces']].unsqueeze(0)

    mse = torch.mean((orgs.view(-1, 3).cpu() - coords.view(-1, 3).cpu())**2)
    total_mse += mse 

    if mse < min_mse: min_mse, min_coords, min_orgs = mse, coords, orgs
    if mse > max_mse: max_mse, max_coords, max_orgs = mse, coords, orgs

    if len(random_samples) <= 30:
        random_samples.append(coords) 
    else:
        all_random_samples.extend([ random_samples])
        random_samples, random_samples_pred = [], []

print(f'MSE AVG: {total_mse / sample_size:.10f}, Min: {min_mse:.10f}, Max: {max_mse:.10f}')    
combind_mesh_with_rows('./mse_rows.obj', all_random_samples)


100%|██████████| 200/200 [00:13<00:00, 14.56it/s]

MSE AVG: 0.0005268551, Min: 0.0000168955, Max: 0.0093237488


In [ ]:
transformer.conditioner.text_models[0].model.to("cuda")
transformer = transformer.to("cuda")

dataset.embed_texts(transformer,1)

In [ ]:
 
torch.cuda.empty_cache()
gc.collect()  
 
batch_size = 2
grad_accum_every =4     
trainer = MeshTransformerTrainer(model = transformer,warmup_steps = 10,grad_accum_every=grad_accum_every,num_train_steps=100, dataset = dataset, 
                                  accelerator_kwargs = {"mixed_precision" : "fp16"}, optimizer_kwargs = { "eps": 1e-7} , 
                                 learning_rate = 1e-3, batch_size=batch_size ,checkpoint_every_epoch = 25) 
loss = trainer.train(35, stop_at_loss = 0.00005)   

In [ ]:
import torch
import random
from tqdm import tqdm 
from meshgpt_pytorch import mesh_render 

min_mse, max_mse = float('inf'), float('-inf')
min_coords, min_orgs, max_coords, max_orgs = None, None, None, None
random_samples, random_samples_pred, all_random_samples = [], [], []
total_mse, sample_size = 0.0, 200

random.shuffle(dataset.data)
autoencoder = autoencoder.to("cuda")
for item in tqdm(dataset.data[:sample_size]):
    item['faces'] = item['faces'].to("cuda")
    item['vertices'] = item['vertices'].to("cuda")
    item['face_edges'] = item['face_edges'].to("cuda")
    codes = autoencoder.tokenize(vertices=item['vertices'], faces=item['faces'], face_edges=item['face_edges']) 

    codes = codes.flatten().unsqueeze(0)
    codes = codes[:, :codes.shape[-1] // autoencoder.num_quantizers * autoencoder.num_quantizers] 
 
    coords, mask = autoencoder.decode_from_codes_to_faces(codes)
    orgs = item['vertices'][item['faces']].unsqueeze(0)

    mse = torch.mean((orgs.view(-1, 3).cpu() - coords.view(-1, 3).cpu())**2)
    total_mse += mse 

    if mse < min_mse: min_mse, min_coords, min_orgs = mse, coords, orgs
    if mse > max_mse: max_mse, max_coords, max_orgs = mse, coords, orgs
 
    if len(random_samples) <= 30:
        random_samples.append((coords, mask)) 
    else:
        all_random_samples.extend([ random_samples])
        random_samples, random_samples_pred = [], []

print(f'MSE AVG: {total_mse / sample_size:.10f}, Min: {min_mse:.10f}, Max: {max_mse:.10f}')
mesh_render.save_rendering(f'.\mse_rows.obj', all_random_samples)

  0%|          | 1/200 [00:00<00:27,  7.33it/s]

100%|██████████| 200/200 [00:11<00:00, 17.40it/s]
/usr/local/lib/python3.10/dist-packages/meshgpt_pytorch/mesh_render.py:8: RuntimeWarning: invalid value encountered in divide
  normal = normal / np.linalg.norm(normal)
/usr/local/lib/python3.10/dist-packages/meshgpt_pytorch/mesh_render.py:19: RuntimeWarning: invalid value encountered in scalar divide
  angle_rad = np.arccos(np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2)))
/usr/local/lib/python3.10/dist-packages/meshgpt_pytorch/mesh_render.py:19: RuntimeWarning: invalid value encountered in arccos
  angle_rad = np.arccos(np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2)))


MSE AVG: 0.0006020016, Min: 0.0000172228, Max: 0.0150029017
[Save_rendering] Saved at .\mse_rows.obj


In [ ]:
now_utc = datetime.datetime.now(datetime.timezone.utc).isoformat()
now_safe = now_utc.replace(":", "_")
filename = f"./MeshGPT-autoencoder_{now_safe}.pt"
pkg = dict( model = autoencoder.state_dict(), ) 
torch.save(pkg, filename)